# memory

'04_memory.ipynb' 

- 대화 내용을 기억 
- LLM은 대화 내용을 기억하지 않는다.(stateless)
- 이전 대화 내용을 계속 프롬프트에 주입해야 함
- 

1. Short-Term : 한 대화 세션에 대한 기억
2. Long-Term : 전체 세션에 대한 중요한 정보 

데이터베이스-디스크 저장
램 저장


# ConversationBufferMemory
- 메시지 저장 -> 변수에서 추출 가능

In [1]:
from langchain.memory import ConversationBufferMemory

memory =  ConversationBufferMemory()
memory.save_context(
    inputs={
        'human': '안녕 나는 비대면 은행 계좌를 개설하고 싶어. 어찌해야함?'
    },
    outputs={
        'ai': '좋아. 우선 신분증을 준비해 줘'
    }
)
print(memory.load_memory_variables({}))

memory.save_context(
    inputs={'human': '좋아. 준비했어'},
    outputs={'ai': '앞면을 잘 촬영해서 업로드 해줘.'}
)
print(memory.load_memory_variables({}))

{'history': 'Human: 안녕 나는 비대면 은행 계좌를 개설하고 싶어. 어찌해야함?\nAI: 좋아. 우선 신분증을 준비해 줘'}
{'history': 'Human: 안녕 나는 비대면 은행 계좌를 개설하고 싶어. 어찌해야함?\nAI: 좋아. 우선 신분증을 준비해 줘\nHuman: 좋아. 준비했어\nAI: 앞면을 잘 촬영해서 업로드 해줘.'}


/var/folders/1p/yb8d7hmx6wd564h57_td4k140000gn/T/ipykernel_23187/1673803115.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory =  ConversationBufferMemory()


In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import ConversationChain

prompt = ChatPromptTemplate.from_message()

# temperature => 랜덤성(0~1)
llm = ChatOpenAI(model='gpt-4.1-nano',temperature=0) # NS 

Conversation = ConversationChain(
    llm = llm,
    memory = ConversationBufferMemory()
)

response = conversation.invoke({''})